In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

os.environ['MOSEKLM_LICENSE_FILE'] = '/content/mosek.lic'

In [4]:
# For Google Colab use, commands installing packages
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

# Install PICOS and CVXOPT in Google Colab
if IN_COLAB:
    !pip install -q picos
    !pip install -q mosek

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.7/510.7 kB 20.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 42.5 MB/s eta 0:00:00


In [5]:
import picos as pic
import numpy as np

In [6]:
print('Solvers supported on this installation of picos:', pic.solvers.all_solvers().keys())
print('Solvers available to picos on this machine :', pic.solvers.available_solvers())

Solvers supported on this installation of picos: dict_keys(['cplex', 'cvxopt', 'ecos', 'glpk', 'gurobi', 'mosek', 'mskfsn', 'osqp', 'scip', 'smcp'])
Solvers available to picos on this machine : ['cvxopt', 'ecos', 'mosek', 'mskfsn', 'osqp']


In [7]:
def P23_matrix(da, db1, db2):
    """
    Create the permutation matrix S_{b1,b2} for dimensions m, n, and p
    such that it permutes the tensor product as:
    S_{b1,b2}(|\psi_{a}⟩ ⊗ |\phi_{b_{1}}⟩ ⊗ |\theta_{b_{2}}⟩) = |\psi_{a}⟩ ⊗ (|\theta_{b_{1}}⟩ ⊗ |\phi_{b_{2}}⟩)

    Parameters:
    da (int): Dimension of |\psi_a⟩
    db1 (int): Dimension of |\phi_{b_1}⟩
    db2 (int): Dimension of |\theta_{b_2}⟩

    Returns:
    numpy.ndarray: The permutation matrix
    """
    # Total size of the tensor product space
    total_size = da * db1 * db2

    # Create the permutation matrix
    P = np.zeros((total_size, total_size))

    for a in range(da):
        for b1 in range(db1):
            for b2 in range(db2):
                old_index = a * db1 * db2 + b1 * db2 + b2
                new_index = a * db1 * db2 + b2 * db1 + b1
                P[new_index, old_index] = 1

    return P

In [8]:
def createPartition(size):
  d = size
  col = 0
  P = np.zeros((d*d,int(d*(d+1)/2)))
  for i in range(d):
      for j in range(i,d):
          P[d*i+j,col] = 1
          P[d*j+i,col] = 1
          col = col+1
  return P

In [9]:
createPartition(4)

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [10]:
# P23 Test
# Define two matrices
phi1 = np.array([[1],
              [2]])
da = 2

phi2 = np.array([[2],
              [3]])
db1 = 2

phi3 = np.array([[4],
              [5]])
db2 = 2

# create permatutation matrix
Sab1b2 = P23_matrix(da, db1, db2)

if np.all(Sab1b2 @ np.kron(np.kron(phi1, phi2), phi3)) == np.all(Sab1b2 @ np.kron(np.kron(phi1, phi3), phi2)):
  print("Permutation matrix is correct")
else:
  print("Permutation matrix is incorrect")

Permutation matrix is correct


# Example 3

Given a bell state $|\phi⟩ = \frac{1}{\sqrt{2}}(|00⟩ + |11⟩)$ check if $\rho_{ab}$ has a $k=2$ extension.

In [11]:
#State to check separability
phiMat = np.array([[1.,0.,0.,1.],
                [0.,0.,0.,0.],
                [0.,0.,0.,0.],
                [1.,0.,0.,1.]])/2

k = 2
da = 2
db = 2
db1 = db
db2 = db
dB = db1*db2

In [12]:
#Constants
#----------
pab = pic.Constant("pab", phiMat)


Ia = pic.Constant('Ia', np.eye(da))
Ib = pic.Constant('Ib', np.eye(db))
Iab = pic.Constant('Iab', np.eye(da*db))
IaB = pic.Constant('IaB', np.eye(da*dB))
Ib1b2 = pic.Constant('Ib1b2', np.eye(db1*db2))
Iab1b2 = pic.Constant('Iab1b2', np.eye(da*db1*db2))

Sb1b2 = pic.Constant('Sb1b2', P23_matrix(da, db1, db2)) # swap
prod = pic.Constant('IIb1b2', (Iab1b2 + Sb1b2)/2) # projector

shpa = (da, da)
shpab = (da*db,da*db)
shpb = (db,db)
shpB = (dB,dB)
shpaB = (da*dB, da*dB)
shpsys = (da,db1,db2)

partition = createPartition(k)

#Variables
#----------
W = pic.HermitianVariable('W', shpab)

The $k=2$ formulation is
$$\max_{W, Z_{j}} \rho_{ab} \boldsymbol{\cdot} W$$
$$\text{ subject to }\prod_{b_{1}b_{2}}(W \otimes I_{b_{2}} + Z)\prod_{b_{1}b_{2}} = 0$$
$$I_{ab} \boldsymbol{\cdot} W = -1$$
$$Z \succeq 0$$

where with $S_{b1,b2}$ defined as
$$S_{b1,b2}(|\psi_{a}⟩ ⊗ |\phi_{b_{1}}⟩ ⊗ |\theta_{b_{2}}⟩) = |\psi_{a}⟩ ⊗ (|\theta_{b_{1}}⟩ \otimes |\phi_{b_{2}}⟩)$$

In [13]:
prob3P = pic.Problem()

#Constraints
#----------
prob3P.add_constraint(pic.trace(Iab * W) == -1)

prob3P.add_constraint((Ia @ partition.conj().T)*(W @ Ib)*(Ia @ partition) << 0)

<6×6 Complex LMI Constraint: Ia⊗[3×4]·(W⊗Ib)·(Ia⊗[4×3]) ≼ 0>

In [14]:
#Objective
#----------
prob3P.set_objective('max', pic.trace(pab*W))

#User readable view of the problem being composed in PICOS'
print(prob3P)

Complex Semidefinite Program
  maximize tr(pab·W)
  over
    4×4 hermitian variable W
  subject to
    tr(Iab·W) = -1
    Ia⊗[3×4]·(W⊗Ib)·(Ia⊗[4×3]) ≼ 0


In [17]:
#Solve the problem using cvxopt as a solver
prob3P.solve(verbosity=True,solver='mosek')

           PICOS 2.4.17            
Problem type: Complex Semidefinite Program.
Strategy outdated:
1. ExtraOptions
2. ComplexLMIToRealReformulation
3. CVXOPTSolver.
Searching a solution strategy for MOSEK via Optimizer API.
Solution strategy:
  1. ExtraOptions
  2. ComplexLMIToRealReformulation
  3. MOSEKSolver
Applying ExtraOptions.
Applying ComplexLMIToRealReformulation.
Building a MOSEK problem instance.
Starting solution search.
-----------------------------------
               MOSEK               
         via Optimizer API         
-----------------------------------
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 79              
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 16              
  Matrix variables       : 1 (scalarized:

<primal feasible solution pair (claimed optimal) from mosek>

In [18]:
print('Status at the end of solving the problem using mosek:', prob3P.status)

mu3P =  prob3P.value

print('Least mu obtained from solving the SDP using mosek above is', mu3P)

Status at the end of solving the problem using mosek: optimal
Least mu obtained from solving the SDP using mosek above is 0.12499998699549014


# Example 4

Check for the existence of PPT symmetric extension with $k=2$ for a two qutrit state
$$p_{ab} = \frac{2}{7}|\psi_{+}⟩⟨\psi_{+}| + \frac{α}{7}\sigma_{+} + \frac{5 -  \alpha}{7}S_{ab}\sigma_{+} + S_{ab}$$
where $|\psi_{+}⟩ = \frac{1}{\sqrt{3}}(|00⟩ + |11⟩ + |22⟩)$, $|\sigma_{+}⟩ = \frac{1}{\sqrt{3}}(|01⟩⟨01| + |12⟩⟨12| + |20⟩⟨20|)$, $S_{ab}$ is the swap operator, and $0 \leq \alpha \leq \frac{5}{2}$

In [19]:
k = 3
da = 3
db = 3
db1 = db
db2 = db
dB = db1*db2

In [20]:
#Re-construct input state from Example 2
idMat = np.eye(da*db)
psiP = idMat[0] + idMat[4] + idMat[8]
psiP = np.outer(psiP,psiP)
psiP = psiP/np.trace(psiP)

In [21]:
sigPlus = np.outer(idMat[1],idMat[1]) + np.outer(idMat[5],idMat[5]) + np.outer(idMat[6],idMat[6])
sigPlus = sigPlus/np.trace(sigPlus)
sigPlusEx = np.outer(idMat[3],idMat[3]) + np.outer(idMat[7],idMat[7]) + np.outer(idMat[2],idMat[2])
sigPlusEx = sigPlusEx/np.trace(sigPlusEx)

In [22]:
alpha = 1.95
rhoMat = (2/7)*psiP + (alpha/7)*sigPlus + ((5-alpha)/7)*sigPlusEx

In [23]:
#Constants
#----------
pab = pic.Constant("pab", rhoMat)
pa = pic.partial_trace(pab, subsystems=(1), dimensions=(db, db))

Ia = pic.Constant('Ia', np.eye(da))
Ib = pic.Constant('Ib', np.eye(db))
IB = pic.Constant('IB', np.eye(dB))
Iab = pic.Constant('Iab', np.eye(da*db))
IaB = pic.Constant('IaB', np.eye(da*dB))
Iab1b2 = pic.Constant('Iab1b2', np.eye(da*db1*db2))

Sb1b2 = pic.Constant('Sb1b2', P23_matrix(da, db1, db2))

prod = pic.Constant('IIb1b2', (Iab1b2 + Sb1b2)/2)

partition = createPartition(k)

shpab = (da*db, da*db)
shpB = (dB, dB)
shpaB = (da*dB, da*dB)
shpsys = (da,db1,db2)

#Initial Points
#---------
Y_init = Ia
W_init = (-1/(db*pic.trace(Y_init)))*Y_init@Ib
Z_init = (1/((db**(k-1)*pic.trace(Y_init)))*Y_init@IB)

#Variables
#----------
W = pic.HermitianVariable('W', shpab)

In [24]:
prob4P = pic.Problem()

#Constraints
#----------
prob4P.add_constraint(pic.trace(Iab * W) == -1)

prob4P.add_constraint((Ia @ partition.conj().T)*(W @ Ib)*(Ia @ partition) << 0)

<18×18 Complex LMI Constraint: Ia⊗[6×9]·(W⊗Ib)·(Ia⊗[9×6]) ≼ 0>

In [25]:
#Objective
#----------
prob4P.set_objective('max', pic.trace(pab*W))

#User readable view of the problem being composed in PICOS'
print(prob4P)

Complex Semidefinite Program
  maximize tr(pab·W)
  over
    9×9 hermitian variable W
  subject to
    tr(Iab·W) = -1
    Ia⊗[6×9]·(W⊗Ib)·(Ia⊗[9×6]) ≼ 0


In [28]:
prob4P.solve(verbosity=True,solver="mosek")

           PICOS 2.4.17            
Problem type: Complex Semidefinite Program.
Strategy outdated:
1. ExtraOptions
2. ComplexLMIToRealReformulation
3. CVXOPTSolver.
Searching a solution strategy for MOSEK via Optimizer API.
Solution strategy:
  1. ExtraOptions
  2. ComplexLMIToRealReformulation
  3. MOSEKSolver
Applying ExtraOptions.
Applying ComplexLMIToRealReformulation.
Building a MOSEK problem instance.
Starting solution search.
-----------------------------------
               MOSEK               
         via Optimizer API         
-----------------------------------
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 667             
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 81              
  Matrix variables       : 1 (scalarized:

<primal feasible solution pair (claimed optimal) from mosek>

In [29]:
print('Status at the end of solving the problem using mosek:', prob4P.status)

mu4P =  prob4P.value

print('Least mu obtained from solving the SDP using mosek above is', mu4P)

Status at the end of solving the problem using mosek: optimal
Least mu obtained from solving the SDP using mosek above is -4.160998710867059e-08
